In [ ]:
# Fetch informations from OntoMe.
# Usually, this cell takes 15 to 19 minutes.
# The slow part are requests of all classes of a profile.

from typing import List, Dict
from requests import get

# Where result will be saved
namespaces = {}
classes = {}
properties = {}

# This function fetches all Profiles created on OntoMe.
# This is needed, because the API only provides properties related to a profile.
# At this moment, there is no way via an API to get all properties from OntoMe.
# Scrap the website would be a possibility, but this way is easier, and easier to maintain.
def fetch_profiles() -> List[Dict]:
    print("> Fetching profiles...", end=" ")

    # Prepare and make the request
    url = "https://ontome.net/api/profiles.json?lang=en"
    response = get(url).json()

    # Parse the answer
    profiles = []
    for element in response:
        profiles.append({"pk": element["profileID"], "label": element["profileLabel"]})

    print("Done")
    return profiles


# As mentioned above, fetch all classes for all profiles.
# The Dict way of doing for results makes sure that everything is unique.
def fetch_classes(profiles: List[Dict]) -> List[Dict]:
    print("> Fetching classes...", end=" ")

    # Loop through all the profiles
    for profile in profiles:

        # Prepare and make the request
        url = f'https://ontome.net/api/classes-profile.json?lang=en&available-in-profile={profile["pk"]}'
        response = get(url).json()

        # Parse the answer
        for element in response:

            # Extract and save the namespace of retrieved class
            namespace = {
                "prefix": element['namespacePrefix'],
                "label": element['namespaceLabel'],
                "uri": element['namespaceURI']
            }
            namespace_key = namespace['prefix']
            namespaces[namespace_key] = namespace

            # Extract and save the retrieved class
            klass = {
                "label": element['classLabel'],
                "id": element['classIdentifierInNamespace'],
                "namespace": element['namespacePrefix']
            }
            klass_key = f"{klass['namespace']}:{klass['id']}"
            classes[klass_key] = klass

    print("Done")


# As mentioned above, fetch all properties for all profiles.
# The Dict way of doing for results makes sure that everything is unique.
def fetch_properties(profiles: List[Dict]) -> List[Dict]:
    print("> Fetching properties...", end=" ")

    # Loop through all the profiles
    for profile in profiles:

        # Prepare and make the request
        url = f'https://ontome.net/api/properties-profile.json?lang=en&available-in-profile={profile["pk"]}'
        response = get(url).json()

        # Parse the answer
        for element in response:

            # Extract and save the namespace of retrieved property
            namespace = {
                "prefix": element['namespacePrefix'],
                "label": element['namespaceLabel'],
                "uri": element['namespaceURI']
            }
            namespace_key = namespace['prefix']
            namespaces[namespace_key] = namespace

            # Extract and save the retrieved property
            property = {
                "label": element['propertyLabel'],
                "id": element['propertyIdentifierInNamespace'],
                "namespace": element['namespacePrefix']
            }
            property_key = f"{property['namespace']}:{property['id']}"
            properties[property_key] = property

    print("Done")


####################################@

# Fetch everything from OntoMe
profiles = fetch_profiles()
fetch_classes(profiles)
fetch_properties(profiles)

> Fetching profiles... Done
> Fetching classes... Done
> Fetching properties... Done


In [ ]:
# Create all python classes.
# Based on retrieved informations (namespaces, classes, and properties),
# save everything correctly: one file is created for each namespace. 
# And for each of those, extract all classes and properties from the whole list.
# Then, save a correclty formated Python class. 
# This new classes all inherit from the Ontology (graphly.schema.ontology) one.

for key, value in namespaces.items():

    # Extract classes and properties of this namespace
    namespace_class = [klass for _, klass in classes.items() if klass['namespace'] == value['prefix']]
    namespace_property = [prop for _, prop in properties.items() if prop['namespace'] == value['prefix']]

    # Compute the Python class name (no spaces, no dashes, no quotes...)
    class_name = value['prefix'].replace(' ', '_').replace('-', '_').replace('\'', '_').replace('(', '_').replace(')', '').upper()

    # Begin the file with a comment that explains that the file is generated
    file_content = "# File automatically generated by python script, from OntoMe API\n"
    file_content += "# Script located at: graphly/scripts/ontologies.ipynb\n"
    file_content += f"\n"

    # Add necessary imports
    file_content += f"from graphly.schema.ontology import Ontology\n"
    file_content += f"from graphly.schema.resource import Resource\n"
    file_content += f"\n"

    # Begin to write the class with its declaration as long as its initialization function
    file_content += f"class {class_name}(Ontology):\n"
    file_content += f"\n"
    file_content += f"    def __init__(self) -> None:\n"
    file_content += f"        super().__init__('{value['label']}', '{value['prefix']}', '{value['uri']}')\n"

    # Append all classes of this namespace
    file_content += "\n"
    file_content += "    # Classes\n"
    for klass in namespace_class:
        # The variable name (eg: "E21_Person")
        var = f"{klass['id']}_{klass['label'].title()}".replace(' ', '_').replace('-', '_').replace('\'', '_').replace('(', '_').replace(')', '')
        if "[" in var: var = var[:var.index("[") - 1] + var[var.index("]") + 1:]

        # Class URI and label, to add to Resource instantiation
        uri = f"{klass['namespace']}:{klass['id']}"
        label = klass['label']

        #Resource instantiation
        file_content += f'    {var} = Resource("{uri}", "{label}", class_uri="owl:Class")\n'

    # Append all properties of this namespace
    file_content += "\n"
    file_content += "    # Properties\n"
    for property in namespace_property:
        # The variable name (eg: "E21_Person")
        var = f"{property['id']}_{property['label'].title()}".replace(' ', '_').replace('-', '_').replace('\'', '_').replace('(', '').replace(')', '')
        if "[" in var: var = var[:var.index("[") - 1] + var[var.index("]") + 1:]

        # Class URI and label, to add to Resource instantiation
        uri = f"{property['namespace']}:{property['id']}"
        label = property['label']

        #Resource instantiation
        file_content += f'    {var} = Resource("{uri}", "{label}", class_uri="owl:Property")\n'

    # Write generated classes in the ontologies folder
    file_name = value['prefix'].replace('-', '_')
    with open(f"../graphly/ontologies/{file_name}.py", 'w') as file:
        file.write(file_content)

    # Clear the module init file
    with open(f"../graphly/ontologies/__init__.py", "w") as file:
        pass

    # And append to the module init file all generated class, so that they are ready for import
    with open(f"../graphly/ontologies/__init__.py", 'a') as file:
        file.write(f"from .{file_name} import {class_name}\n")
        file.write(f"{file_name} = {class_name}()\n")
        file.write("\n")